<a href="https://colab.research.google.com/github/rabiaozdemir/Transfer_Learning/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Model,Sequential
from keras.applications import xception, vgg16, mobilenet
from keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D,GlobalAveragePooling2D
from sklearn.metrics import classification_report, confusion_matrix
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount("/content/drive")


In [ ]:
train_path="./drive/MyDrive/chest_xray/train"
test_path="./drive/MyDrive/chest_xray/test"

trainDataGen=ImageDataGenerator(rescale=1./255, validation_split=0.1)
testDataGen=ImageDataGenerator(rescale=1./255)

trainGen=trainDataGen.flow_from_directory(
    train_path,
    target_size=(224,224),
    color_mode='rgb',
    class_mode="categorical",
    batch_size=32,
    subset="training"
    )

valGen=trainDataGen.flow_from_directory(
    train_path,
    target_size=(224,224),
    color_mode='rgb',
    class_mode="categorical",
    batch_size=32,
    subset="validation"
)

testGen=testDataGen.flow_from_directory(
    test_path,
    target_size=(224,224),
    color_mode='rgb',
    class_mode="categorical",
    batch_size=1
    )

Found 4711 images belonging to 3 classes.
Found 521 images belonging to 3 classes.
Found 624 images belonging to 3 classes.


In [ ]:
model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224,224, 3))
print (model.summary())

58900480/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [ ]:
x=model.output
x=Flatten()(x)
x=Dense(32,activation='relu')(x)
x=Dense(32,activation='relu')(x)
x=Dense(256,activation='relu')(x)
#x=Dense(256,activation='relu')(x)
#x=Dense(128,activation='relu')(x)
#x=Dropout(0.2)(x)
output=Dense(3,activation='softmax')(x)

In [ ]:

# tüm layerları donduralım
for layers in model.layers:
    layers.trainable=False


In [ ]:
New_Model=Model(model.input,output)

In [ ]:
print (New_Model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
New_Model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=["accuracy"])
history=New_Model.fit(
 trainGen,
 steps_per_epoch=trainGen.samples//32,
 epochs=10,
 validation_data=valGen,
 validation_steps=valGen.samples//32,
 #shuffle=True,
 #callbacks=[checkpoint]
 )

Epoch 1/10
147/147 [==============================] - 2373s 16s/step - loss: 1.1066 - accuracy: 0.4506 - val_loss: 0.7405 - val_accuracy: 0.7266
Epoch 2/10
147/147 [==============================] - 2305s 16s/step - loss: 0.7882 - accuracy: 0.6776 - val_loss: 0.6431 - val_accuracy: 0.7266
Epoch 3/10
147/147 [==============================] - 2302s 16s/step - loss: 0.6579 - accuracy: 0.7145 - val_loss: 0.5526 - val_accuracy: 0.7227
Epoch 4/10
147/147 [==============================] - 2300s 16s/step - loss: 0.5898 - accuracy: 0.7137 - val_loss: 0.5290 - val_accuracy: 0.7207
Epoch 5/10
147/147 [==============================] - 2295s 16s/step - loss: 0.5339 - accuracy: 0.7269 - val_loss: 0.4881 - val_accuracy: 0.7422
Epoch 6/10
147/147 [==============================] - 2290s 16s/step - loss: 0.5030 - accuracy: 0.7385 - val_loss: 0.4517 - val_accuracy: 0.7500
Epoch 7/10
147/147 [==============================] - 2291s 16s/step - loss: 0.4512 - accuracy: 0.7591 - val_loss: 0.4700 - val_ac

In [ ]:
print("Test Loss : ",score[0])
print("Test Accuracy % : ",score[1]*100)

In [ ]:
y_pred=np.argmax(New_Model.predict(testGen), axis=1)

In [ ]:
print('Confusion Matrix')
print(confusion_matrix(testGen.classes,y_pred))

In [ ]:
print('Classification Report')
target_names=['bacteria_pneumonia', 'normal', 'virus_pneumonia']
print(classification_report(testGen.classes, y_pred, target_names=target_names))